In [4]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import pandas as pd
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

np.random.seed(42)

In [15]:
trade = pd.read_csv('trade_add.csv')
stocks = pd.read_csv('stocks_add.csv')

In [16]:
trade=trade.drop("Unnamed: 0", axis=1)
stocks=stocks.drop("Unnamed: 0", axis=1)

In [9]:
stocks.columns

Index(['기준일자', '종목번호', '종목명', '20년7월TOP3대상여부', '시장구분', '표준산업구분코드_소분류', '거래량',
       '주식가격_만원단위', '종목고가 - 종목저가/종목시가', '주식발행수', '시가총액', 'year', 'month',
       'day', '기준년월'],
      dtype='object')

In [10]:
trade.columns

Index(['기준년월', '그룹번호', '그룹내고객수', '종목번호', '그룹내_매수여부', '그룹내_매도여부', '매도고객수',
       '평균매수수량', '평균매도수량', '매수가격_중앙값', '매도가격_중앙값',
       '매도가격_중앙값 - 매수가격_중앙값/매수가격_중앙값', '매수수량 - 매도수량', '배당률', '거래량',
       '주식가격_만원단위', '종목고가 - 종목저가/종목시가', '시가총액', '매수고객수'],
      dtype='object')

In [17]:
trade=trade[["기준년월",'그룹번호',"매수고객수","종목번호","매수고객수"]]
stocks=stocks[['기준일자', '종목번호','시장구분','표준산업구분코드_소분류', '거래량',
       '종목고가 - 종목저가/종목시가', '주식발행수', '시가총액', 'year', 'month','day']]

In [28]:
trade['month']=trade['기준년월']%100

In [29]:
trade.head()

,기준년월,그룹번호,매수고객수,종목번호,매수고객수,day,month
0,201907,MAD01,7,A000660,7,7,7
1,201907,MAD04,23,A000660,23,7,7
2,201907,MAD05,3,A000660,3,7,7
3,201907,MAD06,27,A000660,27,7,7
4,201907,MAD09,7,A000660,7,7,7


In [22]:
stocks.head()

,기준일자,종목번호,시장구분,표준산업구분코드_소분류,거래량,종목고가 - 종목저가/종목시가,주식발행수,시가총액,year,month,day
0,20190701,A000020,1,의약품 제조업,31341,1.652893,27931470.0,2.725415e+07,2019,7,201
1,20190701,A000080,1,알코올음료 제조업,127186,1.703163,70133611.0,1.425628e+08,2019,7,201
2,20190701,A000100,1,의약품 제조업,395757,6.756757,66856810.0,1.712882e+09,2019,7,201
3,20190701,A000120,1,도로 화물 운송업,62513,2.930403,22812344.0,3.087602e+08,2019,7,201
4,20190701,A000140,1,기타 금융업,7661,3.090508,23206765.0,2.088942e+07,2019,7,201


In [48]:
import category_encoders as ce

#Create object for binary encoding
encoder= ce.BinaryEncoder(cols=['표준산업구분코드_소분류'], return_df=True)

#Fit and Transform Data 
data_encoded=encoder.fit_transform(stocks) 
stocks=data_encoded

In [49]:
MAD01=trade[trade['그룹번호']=='MAD01']

In [50]:
MAD01_merge=MAD01.merge(stocks, on=['month','종목번호'], how='right')
MAD01_merge

,기준년월,그룹번호,매수고객수,종목번호,매수고객수,day_x,month,기준일자,시장구분,표준산업구분코드_소분류_0,...,표준산업구분코드_소분류_4,표준산업구분코드_소분류_5,표준산업구분코드_소분류_6,표준산업구분코드_소분류_7,거래량,종목고가 - 종목저가/종목시가,주식발행수,시가총액,year,day_y
0,201907.0,MAD01,7.0,A000660,7.0,7.0,7,20190701,1,0,...,1,0,0,1,3538183,1.830986,728002365.0,5.118235e+09,2019,201
1,201907.0,MAD01,7.0,A000660,7.0,7.0,7,20190702,1,0,...,1,0,0,1,2495399,3.730273,728002365.0,5.161186e+09,2019,201
2,201907.0,MAD01,7.0,A000660,7.0,7.0,7,20190703,1,0,...,1,0,0,1,2790684,3.797468,728002365.0,5.084828e+09,2019,201
3,201907.0,MAD01,7.0,A000660,7.0,7.0,7,20190704,1,0,...,1,0,0,1,1503707,2.011494,728002365.0,5.096293e+09,2019,201
4,201907.0,MAD01,7.0,A000660,7.0,7.0,7,20190705,1,0,...,1,0,0,1,2043271,3.698435,728002365.0,5.043109e+09,2019,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286056,NaN,NaN,NaN,A353810,NaN,NaN,7,20200722,0,0,...,1,0,1,0,139723,8.506616,2286728.0,1.243553e+07,2020,202
286057,NaN,NaN,NaN,A353810,NaN,NaN,7,20200723,0,0,...,1,0,1,0,110583,7.441016,2286728.0,1.264898e+07,2020,202
286058,NaN,NaN,NaN,A353810,NaN,NaN,7,20200724,0,0,...,1,0,1,0,97763,4.247788,2286728.0,1.292460e+07,2020,202
286059,NaN,NaN,NaN,A353810,NaN,NaN,7,20200727,0,0,...,1,0,1,0,358288,17.277487,2286728.0,1.385830e+07,2020,202


In [54]:
df=MAD01_merge.dropna(axis=0, thresh=20)

In [57]:
df=df.reset_index()
df["datetime"] = pd.to_datetime(df["기준일자"], format='%Y%m%d')
df=df[["datetime", '그룹번호', '매수고객수', '종목번호', 
       '시장구분', '표준산업구분코드_소분류_0', '표준산업구분코드_소분류_1', '표준산업구분코드_소분류_2',
       '표준산업구분코드_소분류_3', '표준산업구분코드_소분류_4', '표준산업구분코드_소분류_5', '표준산업구분코드_소분류_6',
       '표준산업구분코드_소분류_7', '거래량', '종목고가 - 종목저가/종목시가', '주식발행수', '시가총액','매수고객수']]

In [ ]:
df[]

In [58]:
MAD01_test=df[df['datetime']==202006]
MAD01_train=df[(df['datetime']<202006)&(df['datetime']>=202002)]

KeyError: '기준년월'

In [20]:
print(MAD01_test.shape, MAD01_train.shape)

(40, 5) (115, 5)


In [ ]:
MAD01_=MAD01.merge(stocks_1, on=['month','종목번호'], how='right')